In [ ]:
import gaps_online as go
import gaps_online.db as db
import re
from glob import glob
import numpy as np
import math
from collections import defaultdict
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path 

In [ ]:
# tofevents = []
# file = "/data1/nextcloud/cra_data/data/2023_nevis/tof/121/Run121_100.2024_03_05-10_01_53.tof.gaps"
# data = go.tof.get_tofpackets(file, filter=go.cxx_api.PacketType.TofEvent)
# evts = [go.cxx_api.TofEvent.from_tofpacket(k) for k in data]
# tofevents.extend(evts)

In [ ]:
pattern = re.compile(r'/calibration/20240305/RB(\d+)_\d{4}_\d{2}_\d{2}-\d{2}_\d{2}_\d{2}\.cali\.tof\.gaps')
calibrations = glob('/data1/nextcloud/cra_data/data/2023_nevis/tof/calibration/20240305/*.cali.tof.gaps')

calib = {}

# Iterate over each file path
for fname in calibrations:
    #print("Processing file:", fname)
    # Search for the pattern in the file path
    match = pattern.search(fname)
    if match:
        print("Match found:", match.group())
        # Extract RB ID from the match
        rbid = match.group(1)
        calib[rbid] = go.cxx_api.RBCalibration.from_file(fname)
    else:
        print("No match found for:", fname)


In [ ]:
calib = go.tof.calibrations.load_calibrations(Path('/data1/nextcloud/cra_data/data/2023_nevis/tof/calibration/2024_03_12/'))

if len(calib.keys()) != 40: 
    missing = 40 -len(calib.keys())
    print(f'Oh no! {missing} calibrations are missing :(')
else: print('Yay! All calibration files were loaded :)')
#new way to load calibration dict

In [ ]:
go.cxx_api.get_tofpackets??

In [ ]:
!ls /data1/nextcloud/cra_data/data/2023_nevis/tof/calibration/20240305

In [ ]:
baseline_v_dict = defaultdict(list)
outlier_baseline_v_dict = defaultdict(list)
baselines = []
outlier_base = []
tofevents = []
fixed_path = "/data1/nextcloud/cra_data/data/2023_nevis/tof/121/"

#data = "/data1/nextcloud/cra_data/data/2023_nevis/tof/121/Run121_1.2024_03_05-09_47_33.tof.gaps"
files = glob(f'{fixed_path}_*.tof.gaps')
print(len(files))



for fname in files:
    data = go.tof.get_tofpackets(fname, filter=go.cxx_api.PacketType.TofEvent)
    evts = [go.cxx_api.TofEvent.from_tofpacket(k) for k in data]

    for ev in evts:
        rbids = ev.get_rbids()
        rbevents = [ev.get_rbevent(k) for k in rbids]

        for event_id in rbevents:
            if event_id.header.rb_id == 0: continue 
        
            #event_id = tofevents[i].get_rbevent(rb)
            str_id = f'{event_id.header.rb_id:02d}'
            if len(str_id) > 2: continue #error where some ids are in the hundreds
            try:
                calibration = calib[str_id]
            except Exception as e:
                print(e)
                continue
            
            voltages = calibration.voltages(event_id)
    
            for channel in event_id.header.get_channels():
                if channel == 8: continue
                if (event_id.get_channel_adc(channel+1))[0] == 0 and event_id.get_channel_adc(channel+1)[20] == 0 and event_id.get_channel_adc(channel+1)[40] == 0:
                    mean_bl = 0
                    info = {'rbid' : str_id, 'channel' : channel, 'baseline_V' : mean_bl}
                    baselines.append(info)
                else:
                    slice = np.array(voltages[channel][10:50])
                    mean_bl = np.mean(slice)
                    info = {'rbid' : str_id, 'channel' : channel, 'baseline_V' : mean_bl}
    
             
                
                
                if np.abs(mean_bl) < 200:
                    baselines.append(info)
                else: 
                    try:
                        event_id.plot(calib=calib[str_id])
                        print(slice)
                        print(event_id.get_channel_adc(channel + 1))
                        plt.show()
                        raise ValueError("Baseline voltage is an outlier")  # You can customize the exception type and message here
                    except ValueError as e:
                        print("Caught ValueError:", e)
                        continue


                    #event_id.plot(calib = calib[str_id])
                    #print(slice)
                    
                    # print(event_id.get_channel_adc(channel+1))
                    # plt.show()
                    # raise 
                    #outlier_baselines.append(info)#seperating outlier 300mV baselines from others, for ease of presenting data
                    # if n_waveforms <= 10:
                    #     high_baseline_wvs.append(event_id)
                    #     n_waveforms +=1
                
                # else: 
                #     baselines.append(info)
                    
for element in baselines:
    r = element['rbid']
    channel1 = element['channel']
    baseline_V = element['baseline_V']

    baseline_v_dict[(r, channel1)].append(baseline_V)

for element in outlier_base:
    r = element['rbid']
    channel1 = element['channel']
    baseline_V = element['baseline_V']
    
    
    outlier_baseline_v_dict[(r, channel1)].append(baseline_V)


            
        
 
    
        

In [ ]:
x = go.tof.calibrations.load_calibrations??

In [ ]:
# pattern = re.compile(r'/calibration/20240305/RB(\d+)_\d{4}_\d{2}_\d{2}-\d{2}_\d{2}_\d{2}\.cali\.tof\.gaps')
# calibrations = glob('/data1/nextcloud/cra_data/data/2023_nevis/tof/calibration/20240305/*.cali.tof.gaps')

# calib = {}

# # Iterate over each file path
# for fname in calibrations:
#     #print("Processing file:", fname)
#     # Search for the pattern in the file path
#     match = pattern.search(fname)
#     if match:
#         print("Match found:", match.group())
#         # Extract RB ID from the match
#         rbid = match.group(1)
#         calib[rbid] = go.cxx_api.RBCalibration.from_califile(fname)
#     else:
#         print("No match found for:", fname)


In [ ]:
print(calib.keys())

In [ ]:
baseline_v_dict = defaultdict(list)
baselines = []
outlier_baselines = []
baseline_outlier_dict = defaultdict(list)

high_baseline_wvs = []
n_waveforms = 0

for i in tqdm(range(36000)):
    rbids = tofevents[i].get_rbids()
    rbevents = [tofevents[i].get_rbevent(k) for k in rbids]

    
    for event_id in rbevents:
        if event_id.header.rb_id == 0: continue
        
        #event_id = tofevents[i].get_rbevent(rb)
        str_id = f'{event_id.header.rb_id:02d}'
        if len(str_id) > 2: continue
        calibration = calib[str_id]
        
        voltages = calibration.voltages(event_id)

        for channel in event_id.header.get_channels():
            if channel == 8: continue
            slice = np.array(voltages[channel][10:50])
            mean_bl = np.mean(slice)
            info = {'rbid' : str_id, 'channel' : channel, 'baseline_V' : mean_bl}
            
            if mean_bl <= -200:
            
                event_id.plot(calib = calib[str_id])
                print(slice)
                
                print(event_id.get_channel_adc(channel+1))
                plt.show()
                raise 
                outlier_baselines.append(info)#seperating outlier 300mV baselines from others, for ease of presenting data
                if n_waveforms <= 10:
                    high_baseline_wvs.append(event_id)
                    n_waveforms +=1
                
            else: 
                baselines.append(info) #just remove if/else if you want to plot them all at same time (and change loop below)
            

for element in baselines:
    r = element['rbid']
    channel1 = element['channel']
    baseline_V = element['baseline_V']

    baseline_v_dict[(r, channel1)].append(baseline_V)

for element in outlier_baselines: #delete outlier_baselines list etc and delete this for if you want to just print all at once
    r = element['rbid']
    channel1 = element['channel']
    baseline_V = element['baseline_V']

    baseline_outlier_dict[(r, channel1)].append(baseline_V)

            
    

In [ ]:
#testing if you can sort files paths simply

outlier_base = []
fixed_path = "/data1/nextcloud/cra_data/data/2023_nevis/tof/138/Run138"

files = glob(f'{fixed_path}_*.tof.gaps')
files = sorted(files)
fi
print(len(files))

In [ ]:
#optimized version 

baseline_v_dict = defaultdict(list)
baselines = []
outlier_base = []
fixed_path = "/data1/nextcloud/cra_data/data/2023_nevis/tof/138/Run138"

files = glob(f'{fixed_path}_*.tof.gaps')
files = sorted(files)
files = files [-200:]

for fname in tqdm(files):
    data = go.tof.get_tofpackets(fname, filter=go.cxx_api.PacketType.TofEvent)
    evts = (go.cxx_api.TofEvent.from_tofpacket(k) for k in data)

    for ev in evts:
        rbids = ev.get_rbids()

        for rb in rbids:
            event_id = ev.get_rbevent(rb)
            if event_id.header.rb_id == 0:
                continue 
            key = event_id.header.rb_id
            str_id = str(key)
            if len(str_id) > 2:
                continue

            try:
                calibration = calib[key]
            except KeyError:
                print(f"Calibration not found for rbid {str_id}")
                continue
            
            voltages = calibration.voltages(event_id)
    
            for channel in event_id.header.get_channels():
                if channel == 8:
                    continue
                
                adc_data = event_id.get_channel_adc(channel + 1)
                if adc_data[0] == 0 and adc_data[20] == 0:
                    mean_bl = 0
                else:
                    slice = np.array(voltages[channel][10:50])
                    mean_bl = np.mean(slice)
    
                info = {'rbid': str_id, 'channel': channel, 'baseline_V': mean_bl}

                if np.abs(mean_bl) < 200:
                    baselines.append(info)
                else: 
                    try:
                        # event_id.plot(calib=calib[key])
                        # print(slice)
                        # print(event_id.get_channel_adc(channel + 1))
                        # plt.show()
                        raise ValueError("Baseline voltage is an outlier")  # You can customize the exception type and message here
                    except ValueError as e:
                        print("Caught ValueError:", e)
                        continue


for element in baselines:
    r = element['rbid']
    channel1 = element['channel']
    baseline_V = element['baseline_V']
    baseline_v_dict[(r, channel1)].append(baseline_V)

for element in outlier_base:
    r = element['rbid']
    channel1 = element['channel']
    baseline_V = element['baseline_V']
    baseline_v_dict[(r, channel1)].append(baseline_V)


In [ ]:
#print(high_baseline_wvs)

event_rb = high_baseline_wvs[0]

event_rb.plot(calib = calib[str(event_rb.header.rb_id)])

In [ ]:
print((baseline_outlier_dict[('15', 2)]))
print(len(baseline_rms_dict))

In [ ]:
#general
for rb, channel in tqdm(sorted(baseline_v_dict.keys())):
    bv = baseline_v_dict[(rb, channel)]
    plt.hist(bv, bins=70)
    plt.title(f' RB {rb}, channel {channel+1} baseline voltages')
    plt.xlabel('Baseline Voltage [mV]')
    plt.ylabel('count')
    plt.yscale('log')
    #plt.xlim(-10, 20)
    plt.show()

In [ ]:
for rb, channel in baseline_outlier_dict: #just far from zero baselines
    bv_out = baseline_outlier_dict[(rb, channel)]
    plt.hist(bv)
    plt.title(f' Rb {rb}, channel {channel+1} baseline voltages')
    plt.xlabel('Outlier baseline voltage [mV]')
    plt.ylabel('count')
    plt.show()

In [ ]:
for rb, channel in baseline_v_dict: #near and far from zero but two figs used, different scaling on plots
    bv = baseline_v_dict[(rb, channel)]
    bv_out = baseline_outlier_dict[(rb, channel)]
    plt.figure()
    plt.hist(bv)
    plt.title(f' RB {rb}, channel {channel+1} baseline voltages near 0')
    plt.xlabel('Voltage [mV]')
    plt.ylabel('count')
    plt.show()

    plt.figure()
    plt.hist(bv_out)
    plt.title(f' RB {rb}, channel {channel+1} baseline voltages far from 0')
    plt.xlabel('Voltage [mV]')
    plt.ylabel('count')
    plt.show()

In [ ]:
#most up-to-date plots, y axis is matched between a channel/RB pair and they are output as subplots


for rb, channel in tqdm(sorted(baseline_v_dict.keys())):
    bv = baseline_v_dict[(rb, channel)]
    bv_out = outlier_baseline_v_dict[(rb, channel)]

    fig, axs = plt.subplots(2, 1, figsize=(7,7))
    
    # Plot histogram for near 0 baseline voltages
    axs[0].hist(bv, bins = 20)
    axs[0].set_title(f'RB {rb}, channel {channel+1} baseline voltages near 0')
    axs[0].set_xlabel('Voltage [mV]')
    axs[0].set_ylabel('count')
    axs[0].set_yscale('log')

    # Plot histogram for far from 0 baseline voltages
    axs[1].hist(bv_out, bins=20)
    axs[1].set_title(f'RB {rb}, channel {channel+1} baseline voltages far from 0')
    axs[1].set_xlabel('Voltage [mV]')
    axs[1].set_ylabel('count')
    axs[1].set_yscale('log')

    # Set the same y-axis limit for both subplots
    y_max = max(axs[0].get_ylim()[1], axs[1].get_ylim()[1])
    axs[0].set_ylim(0, y_max)
    axs[1].set_ylim(0, y_max)
    
    plt.tight_layout()
plt.show()